In [ ]:
#@title Only run this code if you need to install perch-hoplite
#@markdown You will likely be asked to restart your runtime, but after restarting, don't need to rerun this block.
!pip install git+https://github.com/google-research/perch-hoplite.git

In [ ]:
# @title Imports
from etils import epath
from IPython.display import display
import ipywidgets as widgets
import numpy as np
from perch_hoplite.agile import colab_utils
from perch_hoplite.agile import embed
from perch_hoplite.agile import source_info
from perch_hoplite.db import brutalism
from perch_hoplite.db import interface

In [ ]:
#@title Mount Google Drive for saving data
#@markdown When you run this, you will be prompted to authenticate to grant permissions for colab to save to your Google Drive.

#@markdown After you grant permissions, you will see a code that you will need to copy and paste in the form output that will be generated below.
import os
from google.colab import drive

drive.mount('/content/drive')

In [ ]:
#@title Create a new folder in Drive (if it doesn't already exist) within your Google drive.
base_dir = '/content/drive/My Drive/'
#@ markdown Name of your new folder in Drive
new_folder_name = 'noaa_demo' #@param

drive_output_directory = base_dir + new_folder_name

try:
  if not os.path.exists(drive_output_directory):
    os.makedirs(drive_output_directory, exist_ok=True)
    print(f'Directory {drive_output_directory} created successfully.')
  else:
    print(f'Directory {drive_output_directory} already exists.')
except OSError as e:
    print("Error:", e)

In [ ]:
@title Configuration { vertical-output: true }

# @markdown Configure the raw dataset and output location(s).  The format is a mapping from
# @markdown a dataset_name to a (base_path, fileglob) pair.  Note that the file
# @markdown globs are case sensitive.  The dataset name can be anything you want.
#
# @markdown This structure allows you to move your data around without having to
# @markdown re-embed the dataset.  The generated embedding database will be
# @markdown placed in the base path. This allows you to simply swap out
# @markdown the base path here if you ever move your dataset.

# @markdown By default we only process one dataset at a time.  Re-run this entire portion [Embed] of the notebook
# @markdown once per dataset.

# @markdown For example, we might set dataset_base_path to '/home/me/myproject',
# @markdown and use the glob '\*/\*.wav' if all of the audio files have filepaths
# @markdown like '/home/me/myproject/site_XYZ/audio_ABC.wav' (e.g. audio files are contained in subfolders of the base directory).


# @markdown 1. Create a unique name for the database that will store the embeddings for the target data.
# @markdown For this example, we use the name of the large audio file, but you can use a different name here.
dataset_name = 'Saipan_A_06_151006_091215'  # @param {type:'string'}
# @markdown 2. Input the filepath for the folder that is containing the input audio files.
dataset_base_path = 'gs://noaa-passive-bioacoustic/pifsc/audio/pipan/saipan/pipan_saipan_06/audio'  #@param {type:'string'}
# @markdown 3. Input the file pattern for the audio files within that folder that you want to embed. Some examples for how to input:
# @markdown - All files in the base directory of a specific type (not subdirectories): e.g. `*.wav` (or `*.flac` etc) will generate embeddings for all .wav files (or whichever format) in the dataset_base_path
# @markdown - All files in one level of subdirectories within the base directory: `*/*.flac` will generate embeddings for all .flac files
# @markdown - Single file: `myfile.wav` will only embed the audio from that specific file.
dataset_fileglob = 'Saipan_A_06_151006_091215.df20.*.flac'  # @param {type:'string'}

# @markdown 4. [Optional] If saving the embeddings database to a new directory, specify here.
# @markdown Otherwise, leave blank - by default the embeddings database output will be saved within
# @markdown dataset_base_path where the audio is located. You do not need to specify db_path unless you want to maintain multiple
# @markdown distinct embedding databases, or if you would like to save the output
# @markdown in a different folder. If your input audio data is accessed
# @markdown from a public URL, we recommend specifying a separate output directory here.
db_subdir = '/agile_Saipan_A_06_151006_091215'  # @param {type:'string'}
db_path = drive_output_directory + db_subdir if db_subdir else None
if not db_path or db_path == 'None':
  db_path = None


# @markdown 5. Choose a supported model to generate embeddings: `perch_8` or `birdnet_v2.3` are most common
# @markdown for birds. Other choices include `surfperch` for coral reefs or
# @markdown `multispecies_whale` for marine mammals.
model_choice = 'surfperch'  #@param['perch_8', 'humpback', 'multispecies_whale', 'surfperch', 'birdnet_V2.3']

# @markdown 6. [Optional] Shard the audio for embeddings. File sharding automatically splits audio files into smaller chunks
# @markdown for creating embeddings. This limits both system and GPU memory usage,
# @markdown especially useful when working with long files (>1 hour).
use_file_sharding = True  # @param {type:'boolean'}
# @markdown If you want to change the length in seconds for the shards, specify here.
shard_length_in_seconds = 75  # @param {type:'number'}

# @markdown We also need to specify the targeted sample rate. -2 will give the target sample rate of the model,
# @markdown -1 will use the target sample rate of the original source audio, and any other number >0 will
# @markdown use that specified rate.
target_sample_rate_hz = -1  # @param {type:'number'}

audio_glob = source_info.AudioSourceConfig(
    dataset_name=dataset_name,
    base_path=dataset_base_path,
    file_glob=dataset_fileglob,
    min_audio_len_s=1.0,
    target_sample_rate_hz=target_sample_rate_hz,
    shard_len_s=float(shard_length_in_seconds) if use_file_sharding else None,
)

configs = colab_utils.load_configs(
    source_info.AudioSources((audio_glob,)),
    db_path,
    model_config_key=model_choice,
    db_key='sqlite_usearch',
)
configs

In [ ]:
#@title Initialize the hoplite database (DB) { vertical-output: true }
global db
db = configs.db_config.load_db()
num_embeddings = db.count_embeddings()

print('Initialized DB located at ', configs.db_config.db_config.db_path)

def drop_and_reload_db(_) -> interface.HopliteDBInterface:
  db_path = epath.Path(configs.db_config.db_config.db_path)
  for fp in db_path.glob('hoplite.sqlite*'):
    fp.unlink()
  (db_path / 'usearch.index').unlink()
  print('\n Deleted previous db at: ', configs.db_config.db_config.db_path)
  db = configs.db_config.load_db()

#@markdown If `drop_existing_db` set to True, when the database already exists and contains embeddings,
#@markdown then those existing embeddings will be erased. You will be prompted to confirm you wish to delete those existing
#@markdown embeddings. If you want to keep existing embeddings in the database, then set to False, which will append the new
#@markdown embeddings to the database.
drop_existing_db = False  #@param {type:'boolean'}

if num_embeddings > 0 and drop_existing_db:
  print('Existing DB contains datasets: ', db.get_dataset_names())
  print('num embeddings: ', num_embeddings)
  print('\n\nClick the button below to confirm you really want to drop the database at ')
  print(f'{configs.db_config.db_config.db_path}\n')
  print(f'This will permanently delete all {num_embeddings} embeddings from the existing database.\n')
  print('If you do NOT want to delete this data, set `drop_existing_db` above to `False` and re-run this cell.\n')

  button = widgets.Button(description='Delete database?')
  button.on_click(drop_and_reload_db)
  display(button)

In [ ]:
#@title Run the embedding { vertical-output: true }
#@markdown This may take approximately 15 minutes to run.

print(f'Embedding dataset: {audio_glob.dataset_name}')

worker = embed.EmbedWorker(
    audio_sources=configs.audio_sources_config,
    db=db,
    model_config=configs.model_config)

worker.process_all(target_dataset_name=audio_glob.dataset_name)

print('\n\nEmbedding complete, total embeddings: ', db.count_embeddings())